In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('./feature/df_preprocess.csv', encoding='utf-8')

df.shape

(2050000, 13)

In [5]:
import lightgbm as lgb
from sklearn.metrics import f1_score

In [8]:
train = df[df.is_val == 0]
val = df[df.is_val == 1]

train.shape
val.shape

(2000000, 13)

(50000, 13)

In [87]:
cv_params =  {
    'objective': 'regression',
    'metric': 'rmse',
#     'min_data_in_leaf': 300,
    'num_leaves': 31,
    'max_depth': 7,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'seed': 2018
    }
    
# [6181]	train's rmse: 0.301404	val's rmse: 0.383193
train_data = lgb.Dataset(train[['dict_len', 'title_rate', 'title_rate_max_sub']], label=train.bayes_label)
val_data = lgb.Dataset(val[['dict_len', 'title_rate', 'title_rate_max_sub']], label=val.bayes_label)

bst = lgb.train(cv_params, train_data, early_stopping_rounds=100, verbose_eval=100, valid_sets=[train_data, val_data], 
                    valid_names=['train', 'val'], num_boost_round=10000)

Training until validation scores don't improve for 100 rounds.
[100]	train's rmse: 0.313702	val's rmse: 0.389206
[200]	train's rmse: 0.311701	val's rmse: 0.388081
[300]	train's rmse: 0.310732	val's rmse: 0.38759
[400]	train's rmse: 0.309881	val's rmse: 0.387152
[500]	train's rmse: 0.309419	val's rmse: 0.386973
[600]	train's rmse: 0.308982	val's rmse: 0.386731
[700]	train's rmse: 0.308492	val's rmse: 0.386471
[800]	train's rmse: 0.307934	val's rmse: 0.386186
[900]	train's rmse: 0.307573	val's rmse: 0.38602
[1000]	train's rmse: 0.307232	val's rmse: 0.385799
[1100]	train's rmse: 0.307031	val's rmse: 0.385676
[1200]	train's rmse: 0.306799	val's rmse: 0.385563
[1300]	train's rmse: 0.306632	val's rmse: 0.385503
[1400]	train's rmse: 0.306249	val's rmse: 0.385275
[1500]	train's rmse: 0.305994	val's rmse: 0.385113
[1600]	train's rmse: 0.305702	val's rmse: 0.384989
[1700]	train's rmse: 0.305505	val's rmse: 0.384913
[1800]	train's rmse: 0.305388	val's rmse: 0.384878
[1900]	train's rmse: 0.305239	

In [89]:
pred = bst.predict(val[['dict_len', 'title_rate', 'title_rate_max_sub']])

In [95]:
f1_score(val.label, pred.round())

0.461889548151529